In [1]:
!pip install jupyter-dash

In [2]:
import requests

url = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/t4-Vy4iOU19i8y6E3Px_ww/spacex-dash-app.py"
response = requests.get(url)

with open(r"C:\Users\Andrew\Desktop\IBM-SpaceX-Capstone\spacex-dash-app.py", "wb") as f:
    f.write(response.content)


In [3]:
import requests

url = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv"
response = requests.get(url)

with open("spacex_launch_dash.csv", "wb") as f:
    f.write(response.content)


In [43]:
from dash import Dash, dcc, html
import pandas as pd
import plotly.express as px
from dash.dependencies import Input, Output

# Load dataset
spacex_df = pd.read_csv("spacex_launch_dash.csv")
spacex_df['Outcome'] = spacex_df['class'].map({0: 'Failure', 1: 'Success'})
launch_sites = spacex_df['Launch Site'].unique()

app = Dash(__name__)

# Layout
app.layout = html.Div(children=[
    html.H1('SpaceX Falcon 9 Launch Dashboard',
            style={'textAlign': 'center', 'color': '#503D36',
                   'font-size': 40}),
    
    dcc.Dropdown(
        id='site-dropdown',
        options=[{'label': 'All Sites', 'value': 'ALL'}] +
                [{'label': site, 'value': site} for site in launch_sites],
        value='ALL',
        placeholder="Select a Launch Site here",
        searchable=True
    ),
    
    dcc.RangeSlider(
        id='payload-slider',
        min=spacex_df['Payload Mass (kg)'].min(),
        max=spacex_df['Payload Mass (kg)'].max(),
        step=1000,
        marks={
            int(spacex_df['Payload Mass (kg)'].min()): str(int(spacex_df['Payload Mass (kg)'].min())),
            int(spacex_df['Payload Mass (kg)'].max()): str(int(spacex_df['Payload Mass (kg)'].max()))
        },
        value=[spacex_df['Payload Mass (kg)'].min(), spacex_df['Payload Mass (kg)'].max()]
    ),
    
    html.Div(id='success-count-output', style={'marginTop': 20, 'fontSize': 18}),
    dcc.Graph(id='success-pie-chart'),
    dcc.Graph(id='success-payload-scatter')
])

# Pie chart + summary callback
@app.callback(
    [Output('success-pie-chart', 'figure'),
     Output('success-count-output', 'children')],
    [Input('site-dropdown', 'value'),
     Input('payload-slider', 'value')]
)
def update_outputs(selected_site, payload_range):
    low, high = payload_range
    filtered_df = spacex_df[(spacex_df['Payload Mass (kg)'] >= low) &
                            (spacex_df['Payload Mass (kg)'] <= high)]
    
    if selected_site == 'ALL':
        fig = px.pie(filtered_df,
                     names='Launch Site',
                     values='class',
                     title=f'Total Success Launches by Site (Payload {low}-{high} kg)',
                     color='Launch Site',
                     color_discrete_map={
                         'KSC LC-39A': 'green',
                         'CCAFS SLC-40': 'red',
                         'CCAFS LC-40': 'blue',
                         'VAFB SLC-4E': 'orange'
                     })
        summary = f"Across all sites (payload {low}-{high} kg), there were {filtered_df['class'].sum()} successful launches out of {len(filtered_df)} total."
    else:
        site_df = filtered_df[filtered_df['Launch Site'] == selected_site]
        counts = site_df['Outcome'].value_counts()
        success_count = counts.get('Success', 0)
        failure_count = counts.get('Failure', 0)
        fig = px.pie(site_df,
                     names='Outcome',
                     title=f'Success vs Failure for {selected_site} (Payload {low}-{high} kg)',
                     color='Outcome',
                     color_discrete_map={'Success': 'green', 'Failure': 'red'})
        summary = f"{selected_site} (payload {low}-{high} kg): {success_count} successes and {failure_count} failures."
    return fig, summary

# Scatter plot callback
@app.callback(
    Output('success-payload-scatter', 'figure'),
    [Input('site-dropdown', 'value'),
     Input('payload-slider', 'value')]
)
def update_scatter(selected_site, payload_range):
    low, high = payload_range
    filtered_df = spacex_df[(spacex_df['Payload Mass (kg)'] >= low) &
                            (spacex_df['Payload Mass (kg)'] <= high)]
    
    if selected_site == 'ALL':
        fig = px.scatter(filtered_df,
                         x='Payload Mass (kg)',
                         y='Outcome',
                         color='Outcome',
                         symbol='Booster Version Category',
                         color_discrete_map={'Success': 'green', 'Failure': 'red'},
                         title=f'Payload vs. Outcome for All Sites (Payload {low}-{high} kg)')
    else:
        site_df = filtered_df[filtered_df['Launch Site'] == selected_site]
        fig = px.scatter(site_df,
                         x='Payload Mass (kg)',
                         y='Outcome',
                         color='Outcome',
                         symbol='Booster Version Category',
                         color_discrete_map={'Success': 'green', 'Failure': 'red'},
                         title=f'Payload vs. Outcome for {selected_site} (Payload {low}-{high} kg)')
    
    fig.update_layout(
        legend_title_text='Outcome & Booster Version',
        legend=dict(
            itemsizing='constant',
            orientation='h',
            yanchor='bottom',
            y=-0.3,
            xanchor='center',
            x=0.5
        )
    )
    return fig

# Run app
app.run(jupyter_mode="inline", port=8051)


![All_Sites](Screenshot184233.png)

![All_Sites](39A.png)

![All_Sites](96.png)

![All_Sites](SLC_4E.png)

![All_Sites](SLC_40.png)

![All_Sites](PayVsOutcome.png)